# Modelo DnCNN

In [ ]:
import numpy as np
import cv2
import glob
from skimage.util import random_noise
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import datetime
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim


# Preprocesamiento

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Crear imagenes con mas ruido

In [ ]:
#Método que hace padding alrededor de las imagenes y hace un resice de ellas
def pad_resize_image(image, target_shape):

    height, width = image.shape[:2]
    target_height, target_width = target_shape

    # Calcular el factor de escala y las nuevas dimensiones
    scale = min(target_width / width, target_height / height)
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Redimensionar la imagen
    resized_image = cv2.resize(image, (new_width, new_height))

    pad_height = target_height - new_height
    pad_width = target_width - new_width
    
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    
    padded_image = cv2.copyMakeBorder(resized_image, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded_image


In [ ]:
def gaussian_filter(image, kernel_size=(5, 5), sigma=4.0):
    return cv2.GaussianBlur(image, kernel_size, sigma)

# Función para aplicar filtrado bilateral
def bilateral_filter(image, d=9, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(image, d, sigma_color, sigma_space)


### Una de las opcciones de preprocesar. Creamos los datos de entrenamiento añadiendo ruido.

In [ ]:
#Cargamos los datos
filepath = "./Datos/*"
paths = glob.glob(filepath)

#Reordenamos los datos para que no esten segudos los datos de mismas dimensones
random.shuffle(paths)
separacion = int((len(paths) * 90)/100)
train_clean = []
train_gaus =  []
test_clean = []
test_gaus = []

#Creamos el size que queremos para las imagenes
target_shape = (400, 400) 

#Recorremos los datos para aplicarles el preprocesado
for i in range(len(paths)):
    
    image = cv2.imread(paths[i])
    image = image / 255.0
    noisy_image = random_noise(image, mode='gaussian', mean=0, var=0.02) 

    padded_image = pad_resize_image(image, target_shape)
    padded_noisy_image = pad_resize_image(noisy_image, target_shape)
    
    if i < separacion:
        train_clean.append(padded_image)
        train_gaus.append(padded_noisy_image)
    else:
        test_clean.append(padded_image)
        test_gaus.append(padded_noisy_image)
        


### Una de las opcciones de preprocesar. Creamos los datos target suavizando la imagen.

In [ ]:
#Cargamos los datos
filepath = "./Datos/*"
paths = glob.glob(filepath)

#Reordenamos los datos para que no esten segudos los datos de mismas dimensones
random.shuffle(paths)
separacion = int((len(paths) * 90)/100)
train_clean = []
train_gaus =  []
test_clean = []
test_gaus = []

#Creamos el size que queremos para las imagenes
target_shape = (400, 400) 

#Recorremos los datos para aplicarles el preprocesado
for i in range(len(paths)):
    
    image = cv2.imread(paths[i])
    image = image / 255.0

    clean_image = gaussian_filter(image)
    
    padded_image = pad_resize_image(clean_image, target_shape)
    padded_noisy_image = pad_resize_image(image, target_shape)
    
    if i < separacion:
        train_clean.append(padded_image)
        train_gaus.append(padded_noisy_image)
    else:
        test_clean.append(padded_image)
        test_gaus.append(padded_noisy_image)
        


## Visualizamos las imágenes

In [ ]:
plt.figure(figsize=(15, 5))

#Mostrar un ejemplo de las imagenes
plt.subplot(1, 4, 1)
plt.imshow(train_clean[0])
plt.axis('off')
plt.title('Train clean')

plt.subplot(1, 4, 2)
plt.imshow(train_gaus[0])
plt.axis('off')
plt.title('Train gaus')



plt.show()

In [ ]:
plt.figure(figsize=(15, 5))


plt.subplot(1, 2, 1)
plt.imshow(test_clean[0])
plt.axis('off')
plt.title('Test clean')

plt.subplot(1, 2, 2)
plt.imshow(test_gaus[0])
plt.axis('off')
plt.title('Test gaus')

## Aplanamos los datos
Esto lo hacemos porque el modelo acepta un solo tensor de datos y no se le puede pasar la lista entera de imágenes 

In [ ]:
train_clean = np.array(train_clean)
train_gaus = np.array(train_gaus)
test_clean = np.array(test_clean)
test_gaus = np.array(test_gaus)



# Creación del modelo

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint,LearningRateScheduler


class DnCnn():
    
    #Iniciamos la clase del modelo
    def __init__(self,num_filters=64,output_channels=3,depth = 20,model_name="DnCnn_TFG1"):
        
        self.num_filters = num_filters
        self.output_channels = output_channels
        self.depth = depth
        self.model_name = model_name
        self.decay_factor = 0

        self.model = self.build_DnCnn()
        self.model.compile(optimizer='adam', loss='mae')
        
        self.model.summary()

    #Funcion que construye el modelo
    def build_DnCnn(self,input_shape=(None,None,3)):
        
        inputs = Input(shape=input_shape)
        
        # Primera capa convolucional con activación ReLU
        output = Conv2D(self.num_filters, (3, 3), padding='same', activation='relu')(inputs)
    
        # Capas intermedias con Conv2D + BatchNormalization + ReLU
        for _ in range(2, self.depth):
            output = Conv2D(self.num_filters, (3, 3), padding='same', use_bias=False)(output)
            output = BatchNormalization()(output)
            output = relu(output)

        # Capa final sin activación
        output = Conv2D(self.output_channels, (3, 3), padding='same', use_bias=False)(output)
    
        # Restamos la entrada de la salida para obtener el resultado final
        output = Subtract()([inputs, output])

        model = Model(inputs=inputs, outputs=output)
        
        return model

    
    def ajuste_learning_rate(self,epochs,lr_inicial):
        
        #calculamos el lr final que buscamos
        final_lr = lr_inicial/1000
        
        #Calculamos cuanto tiene que bajar cada vez
        #Explicacion de la linea: 
        #(a donde quiero llegar / donde empiezo) elevado (cuannto avanzo por paso/en cuantos pasostengo para hacerlo)
        self.decay_factor = (final_lr / lr_inicial) ** (1.0 / epochs)


    def lr_scheduler(self,epoch, lr):
        #se calcula el learning rate para la siguiente época
        new_lr = lr * self.decay_factor
        return new_lr

    
    
    def train(self, X_train, y_train, batch_size,epoch, lr):
        # Callbacks
        tensorboard_cb = TensorBoard(log_dir='./logs8/fit',
        histogram_freq=1,
        write_graph=True,
        write_images=True,
        update_freq="epoch"
        )

        path = os.path.join("./checkpoints/" + self.model_name +".keras")
        checkpoint_cb = ModelCheckpoint(filepath=path, save_best_only=True)
        
        self.ajuste_learning_rate(epoch,lr)
        
        # Creamos el callback de LearningRateScheduler
        lr_callback = LearningRateScheduler(self.lr_scheduler) 

        
        # Entrenamos el modelo
        history = self.model.fit(X_train, y_train,
               batch_size=batch_size,
               epochs=epoch,
               callbacks=[tensorboard_cb, checkpoint_cb, lr_callback],
               validation_split=0.2)

        print("Entrenamiento finalizado.")
        return history

    def save(self,nombre):
        self.model.save(nombre)
    
    

In [ ]:
#Inicializamos el modelo
model = DnCnn(model_name = "DnCNN_TFG_prueba_gaus_mae");

# Creamos el modelo

In [ ]:

history = model.train(train_gaus,train_clean,16,10,0.001)
model.save("DnCNN_TFG_prueba_gaus_mae.h5")

# Visualizamos como ha ido el entrenamiento

In [ ]:
from csbdeep.utils import plot_history
print(sorted(list(history.history.keys())))
plt.figure(figsize=(16,5))
plot_history(history,['loss','val_loss']);

# Visualizacion con Tensorboard

In [ ]:
%tensorboard --logdir logs7/fit

# Prediccion de resultado

In [ ]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Cargar el modelo entrenado
model = tf.keras.models.load_model("DnCNN_TFG_prueba_gaus_mae.h5")

# Suponiendo que test_gaus[0] es una sola imagen
noisy =test_gaus[9][np.newaxis, ...]

# Realizar predicciones
predictions = model.predict(noisy)

predicted_image = np.squeeze(predictions, axis=0)

plt.figure(figsize=(15, 5))

# Imagen original con ruido
plt.subplot(1, 2, 1)
plt.title('Imagen con Ruido')
plt.imshow(test_gaus[9])
plt.axis('off')

# Imagen original limpia
plt.subplot(1, 2, 2)
plt.title('Imagen Limpia')
plt.imshow(predicted_image)
plt.axis('off')

plt.show()


In [ ]:
# Función para calcular PSNR y SSIM
def calcular_metricas(original, denoised):
    psnr_value = psnr(original, denoised, data_range=original.max() - original.min())
    ssim_value = ssim(original, denoised, data_range=original.max() - original.min(), channel_axis=2, win_size=3)
    return psnr_value, ssim_value

In [ ]:
psnr_base, ssim_base = calcular_metricas(test_gaus[9],predicted_image)

In [ ]:
print("PSNR: " + str(psnr_base))
print("ssim_base: " + str(ssim_base))